In [3]:
from dataloader import DataLoader
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf
import numpy as np

In [13]:
dl = DataLoader()
X_train, y_train, X_test, y_test = dl.load_mnist()
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [14]:
def convert_labels_to_binary(labels):
    return np.where(labels % 2 == 0, 0, 1)


y_train_binary = convert_labels_to_binary(y_train)
y_test_binary = convert_labels_to_binary(y_test)

print("First 10 binary labels in training set:", y_train_binary[:10])
print("First 10 binary labels in test set:", y_test_binary[:10])

First 10 binary labels in training set: [1 0 0 1 1 0 1 1 1 0]
First 10 binary labels in test set: [1 0 1 0 0 1 0 1 1 1]


In [14]:

# Define the policy network
def build_policy_network():
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(2, activation='softmax')  # Outputs the probability of being positive or negative
    ])
    return model

# Define the classifier
def build_classifier():
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(1, activation='sigmoid')  # Binary classifier
    ])
    return model

policy_network = build_policy_network()
classifier = build_classifier()

C:\Users\FH-Deg\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [16]:
# Compile the models
policy_network.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



def train_policy_network(policy_network, classifier, X_train, y_train_binary):
    # Parameters
    learning_rate = 1e-3
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    with tf.GradientTape() as tape:
        # 1. Use the policy network to predict actions (pseudo-labels) for the training data
        actions = policy_network(X_train, training=True)

        # 2. Convert actions to pseudo-labels. This might involve taking the argmax if your actions are categorical.
        pseudo_labels = tf.argmax(actions, axis=1)

        # 3. Combine true labels and pseudo-labels
        # For simplicity, we're treating all data as unlabeled here; you might have a more sophisticated method
        combined_labels = tf.where(y_train_binary >= 0, y_train_binary, pseudo_labels)

        # 4. Train the classifier on the combined dataset and calculate loss
        classifier_predictions = classifier(X_train, training=True)
        # Squeeze the predictions to match the target's shape
        classifier_predictions = tf.squeeze(classifier_predictions, axis=-1)
        classifier_loss = tf.keras.losses.binary_crossentropy(combined_labels, classifier_predictions)

    # Calculate gradients with respect to policy network's parameters
    gradients = tape.gradient(classifier_loss, policy_network.trainable_variables)
    # Check if gradients exist
    if any(grad is None for grad in gradients):
        raise ValueError("No gradients found. Check the computational graph.")

    # Apply gradients to update the policy network
    optimizer.apply_gradients(zip(gradients, policy_network.trainable_variables))


# Training loop
epochs = 10  # Example number of epochs

for epoch in range(epochs):
    # Train the classifier on the original labeled data
    classifier.fit(X_train, y_train_binary, epochs=1, batch_size=32, verbose=1)
    
    # Update the policy network based on the classifier's performance
    train_policy_network(policy_network, classifier, X_train, y_train_binary)

    print(f'Epoch {epoch+1}/{epochs} completed.')

# Test the classifier
test_loss, test_acc = classifier.evaluate(X_test, y_test_binary, verbose=2)
print(f'\nTest accuracy: {test_acc}')


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9649 - loss: 0.1009


ValueError: No gradients provided for any variable.

Generate python code to build custom interactive algorithm for PU learning.
describe in this paper, Learning Classifiers on Positive and Unlabeled Data
with Policy Gradient
Tianyu Li∗
, Chien-Chih Wang∗
, Yukun Ma†
, Patricia Ortal∗
, Qifang Zhao∗
, Bjorn Stenger ¨
∗
, Yu Hirate∗
∗Rakuten Institute of Technology, Tokyo
†AIR Labs, Continental Automotive Group, Singapore

provide me all steps full code don't miss or assume anything.
steps:
load mnist dataset 
convert all even (0,2,4,6,8) labels class= 0 and odd (1,3,5,7,9) into class= 1
randomly select few class =1 and make it class =0 (rembember we want to make our dataset PU data and considering 0 as unknow and 1 as positive dataset)
next steps I'll provide you in next prompt.
Now provide me code for next steps and both algos will to interactive learning.
1:Initilize a neural network classifer and per-train for few epochs. (later this classifer will help to generate reward)
2:Initilize reinforcement learning algo(Policy-Based Algorithms) with policy gradient.
(we will feed PU data to policy gradient)
Action: agent will take action and select the estimated label (either it's 0 or 1) for each input.
Goal: Maximizes reward from classifier
Reward: after action from RL, classifer will then take estimated lables (actions) and data instances and produce reward for policy gradient. Description for Reward given below.
threshhold =0.5
3: because we made PU data out of reguler mnist dataset, we have test dataset with actual class 0 and 1, so make a accuracy or any suitable matrix to eveluate.

Reward description:

I'm also providing you actual sudo code given in paper you can try to unterstand that as well, further if you have any question please first ask me then generate code.






# PU learning Paper algorithm Impelementation

In [4]:
import numpy as np
from tensorflow.keras.datasets import mnist

# Loading the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Converting labels - Even numbers to 0, Odd numbers to 1 to make it binary class classification
y_train = np.where(y_train % 2 == 0, 0, 1)
y_test = np.where(y_test % 2 == 0, 0, 1)

# Generating PU data by randomly selecting some positive labels and make class=0 consider as unknow
def generate_pu_data(labels, flip_prop=0.2):
    positive_indices = np.where(labels == 1)[0]
    num_to_flip = int(len(positive_indices) * flip_prop)
    flip_indices = np.random.choice(positive_indices, size=num_to_flip, replace=False)
    labels[flip_indices] = 0
    return labels


y_train_pu = generate_pu_data(y_train.copy())
# y_test_pu = generate_pu_data(y_test.copy())


print("Training labels distribution:", np.bincount(y_train_pu))


Training labels distribution: [35593 24407]


Light GBM from microsoft

In [5]:
def build_classifier():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

classifier = build_classifier()
# Pre-training the classifier
classifier.fit(x_train, y_train_pu, epochs=5) # need to read pre-trainging is with PU dataset or with actual dataset

Epoch 1/5


C:\Users\FH-Deg\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.7785 - loss: 5.1998
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8367 - loss: 0.3800
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8433 - loss: 0.3637
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8443 - loss: 0.3596
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8463 - loss: 0.3582


In [6]:
def reset_environment(x_data):
    """
    Returns:
    - state: A randomly selected initial state from the PU dataset.
    """
    initial_index = np.random.randint(0, len(x_data))
    state = x_data[initial_index].flatten()

    return state


def environment_step(action, current_index, x_data, y_data):
    next_index = current_index + 1
    done = next_index >= len(x_data)
    next_state = x_data[next_index].flatten() if not done else None

    return next_state, done

In [10]:
# Building policy gradient-based RL agent
class PolicyGradientAgent:
    def __init__(self, input_shape, action_space):
        self.action_space = action_space
        self.model = self.build_policy_model(input_shape, action_space)
    
    def build_policy_model(self, input_shape, action_space):
        model = tf.keras.Sequential()
        
        # This need to configured for each data set
        model.add(tf.keras.layers.Dense(128, activation='relu', input_shape=(input_shape,)))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Dense(64, activation='relu'))
        model.add(tf.keras.layers.Dense(action_space, activation='softmax'))
        
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        return model

    def act(self, state):
        # state = state.reshape(-1, *state.shape)
        action_probs = self.model.predict(state)
        action = np.random.choice(self.action_space, p=action_probs[0])
        return action

    def train(self, states, actions, rewards, gamma=0.99):
        states = np.vstack(states)
        actions = np.array(actions)
        rewards = np.array(rewards)

        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(len(rewards))):
            running_add = running_add * gamma + rewards[t]
            discounted_rewards[t] = running_add

        
        discounted_rewards -= np.mean(discounted_rewards)
        discounted_rewards /= np.std(discounted_rewards)

        # Compute loss and gradients
        with tf.GradientTape() as tape:
            probs = self.model(states, training=True)
            action_masks = tf.one_hot(actions, self.action_space)
            filtered_probs = tf.reduce_sum(action_masks * probs, axis=1)
            loss = -tf.reduce_sum(tf.math.log(filtered_probs) * discounted_rewards)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))


def reward_function(classifier_output, action, threshold=0.5):
    
    prob_positive = classifier_output[0, 1]  

    if action == 1:  
        return prob_positive
    else:  
        return 1 - prob_positive if prob_positive < threshold else prob_positive


input_shape = 784  
action_space = 2 
agent = PolicyGradientAgent(input_shape=input_shape, action_space=action_space)



In [12]:
def train_agent(num_episodes, x_data, y_data, agent):
    for episode in range(num_episodes):
        states = []
        actions = []
        rewards = []
        state = reset_environment(x_data)  # Reset environment to get the initial state
        current_index = np.random.randint(0, len(x_data))  # Start from a random index in the dataset
        done = False

        while not done:
            state = state.reshape(1, -1).astype('float32') / 255.0  # Normalize and reshape the state
            action = agent.act(state)  # Agent takes action based on the current state
            states.append(state.flatten())  # Store the flattened state
            actions.append(action)

            next_state, done = environment_step(action, current_index, x_data, y_data)

            if not done:
                # next_state = next_state.reshape(1, -1).astype('float32') / 255.0  # Normalize and reshape the next state
                # classifier_output = classifier.predict(next_state)
                # reward = reward_function(classifier_output, action)
                # state = next_state  # Update the state for the next iteration
                next_state_reshaped = next_state.reshape(1, 28, 28).astype('float32') / 255.0  # Normalize the data
                classifier_output = classifier.predict(next_state_reshaped)  # Use the reshaped state for prediction
                reward = reward_function(classifier_output, action)
                state = next_state
                current_index += 1  # Move to the next index in the dataset
            else:
                reward = 0  # Assign a terminal reward

            rewards.append(reward)

        print(f"Training agent for episode {episode + 1}")
        agent.train(states, actions, rewards)  # Train the agent at the end of the episode

train_agent(num_episodes=2, x_data=x_train, y_data=y_train_pu, agent=agent)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━

In [20]:
# def train_agent(num_episodes, x_data, y_data, agent):
#     for episode in range(num_episodes):
#         states = []
#         actions = []
#         rewards = []
#         state = reset_environment(x_data)
#         current_index = 0  
#         done = False

#         while not done:
#             state = state.flatten() if state.ndim > 1 else state

#             # Agent takes action based on the current state
#             action = agent.act(state)
#             states.append(state)
#             actions.append(action)

#             # Environment responds with the next state and a done flag
#             next_state, done = environment_step(action, current_index, x_data, y_data)
#             next_state = next_state.reshape(1, -1).astype('float32')

#             # Here, you would typically calculate reward based on action and next_state
#             # and update your agent accordingly. This part is omitted for brevity.
#             # Compute the reward for the current action
#             if not done:
#                 next_state_reshaped = next_state.reshape(1, 28, 28)

#                 # OR if your classifier expects (28, 28, 1) shaped inputs (with a channel dimension)
#                 next_state_reshaped = next_state.reshape(1, 28, 28, 1)

#                 # Normalize if your model was trained on normalized data
#                 next_state_reshaped /= 255.0
#                 classifier_output = classifier.predict(next_state_reshaped)  # Assuming the classifier expects flattened, normalized images
#                 reward = reward_function(classifier_output, action)
#             else:
#                 reward = 0  # Or some other terminal reward

#             rewards.append(reward)

#             # Update state and current_index for the next iteration
#             state = next_state
#             current_index += 1

#             if done:
#                 print("Trainging agent for current eprisote")
#                 # Train the agent with the accumulated states, actions, and rewards for the episode
#                 agent.train(states, actions, rewards)

            

# train_agent(num_episodes=2, x_data=x_train, y_data=y_train_pu, agent=agent)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━

In [ ]:
# Step 5: Evaluation
# Evaluate the performance using the test set with actual labels